<a href="https://colab.research.google.com/github/juyvsa/SlugSpot/blob/main/SlugSpot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import folium

# The origin is the McHenry Library
origin = [36.99572839420908, -122.05900508770912]
offset = 0.02

map = folium.Map(
    max_bounds=True,
    location=origin,
    zoom_start=15,
    min_lat=origin[0]-offset,
    max_lat=origin[0]+offset,
    min_lon=origin[1]-offset,
    max_lon=origin[1]+offset,
)

In [ ]:
from datetime import datetime
import pytz

class StudySpot:
    """stores study spot attributes"""
    def __init__(self, name, location, isquiet, isinside, hours):
        self.name = name
        self.location = location
        self.isquiet = isquiet
        self.isinside = isinside
        self.hours = hours

    def __repr__(self):
        return self.name


    def isopen(self):
        """checks if the study spot is currently open"""
        timezone = pytz.timezone("US/Pacific")
        now_pst = datetime.now(timezone)
        time_hours = list(hours_to_time(self.hours))
        if len(time_hours) == 0:  # open 24/7
            return True
        for interval in time_hours:  # interval is list of start and end time
            if interval[0] <= now_pst.time() <= interval[1]:
                return True
        return False

    def qualifies(self, quietness, insideness):
        """checks if study spot is open and fits desired attributes"""

        if 2 > quietness != self.isquiet:
            return False
        if 2 > insideness != self.isinside:
            return False
        return True

In [ ]:
import csv

with open("LocationsUpdated.csv", "r") as f:
  reader = csv.reader(f)
  data = []
  for row in reader:
    data.append(StudySpot(row[0], row[1], row[2] == "Quiet", row[3] == "In", row[4]))
f.close()

In [ ]:
def hours_to_time(hour_string):
    """converts string in csv to list of time intervals"""
    if hour_string == "":
        return
    for interval in hour_string.split():
        times = []
        for hr in interval.split("-"):
            times.append(datetime.strptime(hr, "%H:%M").time())
        yield times

# dict for possible quietness user inputs
quiet_loud = {
    "loud": 0,
    "quiet": 1,
    "either": 2
    }

# dict for possible insideness user inputs
in_out = {
    "outside": 0,
    "inside": 1,
    "either": 2
    }

# dict for main locations and their entries in the csv file
locs = {
    "baskin": [0, 5],
    "nine/jrl": [5, 10],
    "cowell/stevenson": [10, 15],
    "crown/merrill": [15, 22],
    "kresge": [22, 26],
    "oakes": [26, 32],
    "porter": [32, 38],
    "quarry": [38, 43],
    "rcc": [43, 46],
    "science hill": [46, 48],
    "mchenry": [48, 50]
    }

def process_input(inpt, options):
    """understands user input using dict of options"""
    inpt = inpt.lower()
    if inpt in options.keys():
      return options[inpt]
    else:
      print(inpt, "is not a valid option.")
      raise ValueError()


def welcome():
    """welcomes user and gets/processes their inputs"""

    print("\033[94mWelcome! Please enter study location:")
    location = input()
    print("\n")
    print("Would you prefer a quiet or loud place? [quiet/loud/either]")
    quietness = process_input(input(), quiet_loud)
    print("\n")
    print("Would you prefer inside or outside? [inside/outside/either]")
    inside = process_input(input(), in_out)
    print("\n")
    return location, quietness, inside


import re

def recognize_loc(s):
    """identifies which location the user wants to access"""

    patterns = [
        ["baskin", re.compile(r"^(?<!not)(engineer|baskin|be)")],
        ["nine/jrl", re.compile(r"^(?<!not)(c|college)?(nine|9|ten|10|jrl|lewis|john)")],
        ["cowell/stevenson", re.compile(r"^(?<!not)(cowell|stevenson)")],
        ["crown/merrill", re.compile(r"^(?<!not)(crown|merrill)")],
        ["kresge", re.compile(r"^(?<!not)(kresge)")],
        ["oakes", re.compile(r"^(?<!not)(oakes)")],
        ["porter", re.compile(r"^(?<!not)(porter)")],
        ["quarry", re.compile(r"^(?<!not)(quarry|bay tree)")],
        ["rcc", re.compile(r"^(?<!not)(rcc|rachel|carson)")],
        ["science hill", re.compile(r"^(?<!not)(science|physical|pbs|earth|marine|bio)")],
        ["mchenry", re.compile(r"^(?<!not)(mchenry|arc|visual|arts)")]]

    for pattern in patterns:
        if re.search(pattern[1], s):
            return pattern[0]

def get_locs(inpt):
    """finds the study study spots in the desired location"""

    x, y = process_input(recognize_loc(inpt).strip().lower(), locs)
    return data[x:y]

In [ ]:
l = welcome()

print("Open study spots near", l[0], "that fit your preferences:")

for spot in get_locs(l[0]):
  if spot.isopen() and spot.qualifies(l[1],l[2]):
    print(spot)
    if len(spot.hours) == 0:
      h = '-'
    else:
      h = spot.hours
    popup = folium.Popup(f"<strong style='font-size: 16px;'>{spot.name}</strong><br>Hours: {h}", max_width=300, style="white-space: pre-line;")
    lat, lon = spot.location.split(",")
    folium.Marker(location=[float(lat), float(lon)], popup=popup).add_to(map)

map

Welcome! Please enter study location:
merrill


Would you prefer a quiet or loud place? [quiet/loud/either]
quiet


Would you prefer inside or outside? [inside/outside/either]
inside


Open study spots near merrill that fit your preferences:
Crown Acad 104
Casa Latina
Crown Library
